In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

## UMAP

In [ ]:
size = 35
swmhau_network_umap, best_swmhau_network_umap, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap


In [ ]:
swmhau_network_umap.groupby(["dimensions",
                             "output_channels",
                             "sig_depth",
                             "num_heads",
                             "num_layers",
                             "ffn_hidden_dim",
                             "learning_rate"]).mean()

In [ ]:
best_swmhau_network_umap

In [ ]:
best_swmhau_network_umap[["dimensions",
                          "output_channels",
                          "sig_depth",
                          "num_heads",
                          "num_layers",
                          "ffn_hidden_dim",
                          "dropout_rate",
                          "learning_rate"]]

In [ ]:
best_swmhau_network_umap["f1"].mean()

In [ ]:
best_swmhau_network_umap["precision"].mean()

In [ ]:
best_swmhau_network_umap["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap["recall_scores"]).mean(axis=0)

## Random Projections

In [18]:
size = 35
swmhau_network_grp, best_swmhau_network_grp, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_kfold_best_model.csv


In [19]:
swmhau_network_grp.groupby(["dimensions",
                            "output_channels",
                            "sig_depth",
                            "num_heads",
                            "num_layers",
                            "ffn_hidden_dim",
                            "dropout_rate",
                            "learning_rate"]).mean()

/tmp/ipykernel_3944524/1948036447.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(["dimensions",


loss  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.437324   
                                                                                      0.0005         0.497362   
                                                                                      0.0010         0.479793   
                                                                         0.5          0.0001         0.400435   
                                                                                      0.0005         0.503324   
                                                                                      0.0010         0.421962   
                                                          (512, 512)     0.1          0.0001         0.397865   
                                                                                      0.0005         0.438234   
                                                                                      0.0010         0.856663   
                                                                         0.5          0.0001         0.397297   
                                                                                      0.0005         0.421027   
                                                                                      0.0010         0.586376   
                                     12        1          (256, 256)     0.1          0.0001         0.425550   
                                                                                      0.0005         0.457671   
                                                                                      0.0010         0.405572   
                                                                         0.5          0.0001         0.333581   
                                                                                      0.0005         0.654582   
                                                                                      0.0010         0.427511   
                                                          (512, 512)     0.1          0.0001         0.413396   
                                                                                      0.0005         0.690202   
                                                                                      0.0010         0.383327   
                                                                         0.5          0.0001         0.357123   
                                                                                      0.0005         0.563043   
                                                                                      0.0010         0.337940   
           12              3         10        1          (256, 256)     0.1          0.0001         0.394287   
                                                                                      0.0005         0.604171   
                                                                                      0.0010         0.342548   
                                                                         0.5          0.0001         0.361919   
                                                                                      0.0005         0.864935   
                                                                                      0.0010         0.568580   
                                                          (512, 512)     0.1          0.0001         0.386000   
                                                                                      0.0005         0.748276   
                                                                                      0.0010         0.503538   
                                                                         0.5          0.0001         0.358638   
                                     

In [20]:
best_swmhau_network_grp

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.388765,0.634890,0.633350,"[0.657117278424351, 0.6095820591233435]",0.633874,"[0.6461267605633803, 0.6216216216216216]",0.633244,"[0.668488160291439, 0.598]",0.187115,0.747331,...,0.001,1,focal,2,False,None,Conv1d,None,concatenation,64
0,0.796810,0.601525,0.599367,"[0.6287744227353464, 0.5699588477366254]",0.600191,"[0.6135181975736569, 0.586864406779661]",0.599404,"[0.644808743169399, 0.554]",0.164889,0.786477,...,0.001,12,focal,2,False,None,Conv1d,None,concatenation,64
0,0.325039,0.655863,0.652833,"[0.6203995793901157, 0.6852659110723627]",0.670625,"[0.7338308457711443, 0.60741885625966]",0.661670,"[0.5373406193078324, 0.786]",0.202457,0.693950,...,0.001,123,focal,2,False,None,Conv1d,None,concatenation,64


In [ ]:
best_swmhau_network_grp.columns

Index(['loss', 'accuracy', 'f1', 'f1_scores', 'precision', 'precision_scores',
       'recall', 'recall_scores', 'valid_loss', 'valid_accuracy', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 'valid_precision_scores',
       'valid_recall', 'valid_recall_scores', 'k', 'dimensions', 'sig_depth',
       'method', 'input_channels', 'output_channels', 'features',
       'standardise_method', 'add_time_in_path', 'num_features',
       'embedding_dim', 'log_signature', 'num_heads', 'num_layers',
       'ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'gamma', 'k_fold', 'n_splits', 'augmentation_type',
       'hidden_dim_aug', 'comb_method', 'batch_size'],
      dtype='object')

In [35]:
best_swmhau_network_grp[["dimensions",
                         "output_channels",
                         "sig_depth",
                         "num_heads",
                         "num_layers",
                         "ffn_hidden_dim",
                         "dropout_rate",
                         "learning_rate"]]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.001
0,15,12,3,10,1,"(512, 512)",0.1,0.001
0,15,12,3,10,1,"(512, 512)",0.1,0.001


In [21]:
best_swmhau_network_grp["f1"].mean()

0.6285163497470242

In [22]:
best_swmhau_network_grp["precision"].mean()

0.6348967814281874

In [23]:
best_swmhau_network_grp["recall"].mean()

0.6314395871281118

In [24]:
np.stack(best_swmhau_network_grp["f1_scores"]).mean(axis=0)

array([0.63543043, 0.62160227])

In [25]:
np.stack(best_swmhau_network_grp["precision_scores"]).mean(axis=0)

array([0.66449193, 0.60530163])

In [26]:
np.stack(best_swmhau_network_grp["recall_scores"]).mean(axis=0)

array([0.61687917, 0.646     ])